In [13]:
import pytorch_lightning as pl
from src import DataModule, Model
import torch
from pytorch_lightning.metrics.functional.classification import accuracy
from tqdm import tqdm
import numpy as np

In [3]:
checkpoint = torch.load('resnet18-val_acc=0.74102.ckpt')
checkpoint['hyper_parameters']

{'lr': 0.0003,
 'batch_size': 256,
 'precision': 16,
 'epochs': 3,
 'subset': 0.1,
 'size': 256}

In [6]:
model = Model.load_from_checkpoint('resnet18-val_acc=0.74102.ckpt')
model.hparams

"batch_size": 256
"epochs":     3
"lr":         0.0003
"precision":  16
"size":       256
"subset":     0.1

In [14]:
def evaluate(model, dl):   
    model.eval()
    model.cuda()
    acc = []
    with torch.no_grad():
        t = tqdm(dl)
        for x, y in t:
            x, y = x.cuda(), y.cuda()
            y_hat = model(x)
            acc.append(accuracy(y_hat, y).item())
            t.set_description(f"acc {np.mean(acc):.5f}")

In [15]:
dm = DataModule()
dm.setup()
evaluate(model, dm.val_dataloader())

  0%|     | 0/67 [00:00<?, ?it/s]

Training samples:  17117
Validation samples:  4280


acc 0.72148: 100%|█| 67/67 [01:25


In [16]:
class Preprocess(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        x = x.float() / 255.
        x = x.permute(0, 3, 1, 2)
        return x 
    
class Postprocess(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return torch.argmax(x, dim=1)

In [17]:
script = torch.jit.script(torch.nn.Sequential(
    Preprocess(),
    model.resnet.cpu(),
    Postprocess()
))
torch.jit.save(script, "model.pt")

In [18]:
def evaluate2(model, dl):   
    model.eval()
    model.cuda()
    acc = []
    with torch.no_grad():
        t = tqdm(dl)
        for x, y in t:
            x, y = x.cuda(), y.cuda()
            # simulate test
            x *= 255. 
            x = x.permute(0, 2, 3, 1).long()
            y_hat = model(x)
            acc.append(accuracy(y_hat, y).item())
            t.set_description(f"acc {np.mean(acc):.5f}")

In [19]:
loaded = torch.jit.load('model.pt')
evaluate2(loaded, dm.val_dataloader())

acc 0.72148: 100%|█| 67/67 [01:17
